In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Imports

In [2]:
from __future__ import print_function
from __future__ import division

import copy
import json
import re
import string

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn  # To improve the chart styling.
import wordtree

from IPython.display import display
from IPython.display import HTML
from IPython.display import Javascript
from wordcloud import STOPWORDS
import ipywidgets as widgets
from wordcloud import WordCloud

# import iphone_connector


# Load the data from disk and set up the dataframes

In [3]:
%matplotlib inline
matplotlib.style.use('ggplot')
pd.set_option('display.max_colwidth', 1000)
# iphone_connector.initialize()



In [4]:
import os

In [5]:
df = pd.read_excel("HansardFlatDatasetUpdated.xlsx")

In [6]:
df.head()

,qTextID,qHeading,talkerID,talkKind,qDate,qNum,houseName,date,sessionName,parliamentNum,...,discussionType,hqTextID,tqTextID,qtalkerID,qtalkKind,textID,talkerRole,talkerName,talkerTranscript,fileName
0,201904304784844826e74509a0000001,Dog Fence,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,201904304784844826e74509a0000001,201904304784844826e74509a0000001,3404,question,201904304784844826e74509a0000004,member,The Hon. D.W. RIDGWAY,1.&#x9;I refer the honourable member to the media release issued on 7 March 2019 titled '$25 Million fix required for Ageing Dog Fence.',HANSARD-10-26496.xml
1,201904304784844826e74509a0000001,Dog Fence,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,201904304784844826e74509a0000001,201904304784844826e74509a0000001,3404,question,201904304784844826e74509a0000005,member,The Hon. D.W. RIDGWAY,"2.&#x9;In total, 115 kilometres of the 2,150 kilometre South Australian Dog Fence was replaced over 16 years under the former Labor government. Of this, 68.5 kilometres is electric fence, which now needs to be replaced because it is too low and because pastoralists don't believe it is effective.",HANSARD-10-26496.xml
2,201904304784844826e74509a0000001,Dog Fence,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,201904304784844826e74509a0000001,201904304784844826e74509a0000001,3404,question,201904304784844826e74509a0000006,member,The Hon. D.W. RIDGWAY,3.&#x9;Based on the advice received about the failure of the electric fencing installed under the former Labor government I would not envisage replacing sections of the Dog Fence with electric fencing.,HANSARD-10-26496.xml
3,2019043047e58af38d714803b0000002,Adelaide Metro,3404,question,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,2019043047e58af38d714803b0000002,2019043047e58af38d714803b0000002,3404,question,2019043047e58af38d714803b0000003,member,The Hon. J.A. DARLEY,In reply to the Hon. J.A. DARLEY (5 December 2018).,HANSARD-10-26492.xml
4,2019043047e58af38d714803b0000002,Adelaide Metro,1820,answer,2019-04-30,NaN,Legislative Council,2019-04-30,"Fifty-Fourth Parliament, First Session (54-1)",54,...,Answers to Questions,2019043047e58af38d714803b0000002,2019043047e58af38d714803b0000002,3404,question,2019043047e58af38d714803b0000004,member,The Hon. D.W. RIDGWAY,"The Minister for Transport, Infrastructure and Local Government has provided the following advice:The Hon. D.W. RIDGWAY (Minister for Trade, Tourism and Investment): The Minister for Transport, Infrastructure and Local Government has provided the following advice:",HANSARD-10-26492.xml


In [7]:
df.columns

Index(['qTextID', 'qHeading', 'talkerID', 'talkKind', 'qDate', 'qNum',
       'houseName', 'date', 'sessionName', 'parliamentNum', 'sessionNum',
       'parliamentName', 'reviewStage', 'dateModified', 'proceeding',
       'discussionType', 'hqTextID', 'tqTextID', 'qtalkerID', 'qtalkKind',
       'textID', 'talkerRole', 'talkerName', 'talkerTranscript', 'fileName'],
      dtype='object')

In [8]:
df = df[['qTextID', 'talkerID', 'talkerTranscript']]

In [9]:
df.head()

,qTextID,talkerID,talkerTranscript
0,201904304784844826e74509a0000001,1820,1.&#x9;I refer the honourable member to the media release issued on 7 March 2019 titled '$25 Million fix required for Ageing Dog Fence.'
1,201904304784844826e74509a0000001,1820,"2.&#x9;In total, 115 kilometres of the 2,150 kilometre South Australian Dog Fence was replaced over 16 years under the former Labor government. Of this, 68.5 kilometres is electric fence, which now needs to be replaced because it is too low and because pastoralists don't believe it is effective."
2,201904304784844826e74509a0000001,1820,3.&#x9;Based on the advice received about the failure of the electric fencing installed under the former Labor government I would not envisage replacing sections of the Dog Fence with electric fencing.
3,2019043047e58af38d714803b0000002,3404,In reply to the Hon. J.A. DARLEY (5 December 2018).
4,2019043047e58af38d714803b0000002,1820,"The Minister for Transport, Infrastructure and Local Government has provided the following advice:The Hon. D.W. RIDGWAY (Minister for Trade, Tourism and Investment): The Minister for Transport, Infrastructure and Local Government has provided the following advice:"


In [10]:
df['Text'] = df['talkerTranscript'].astype(str) 

In [11]:
subdf = df[0:1000]

In [13]:
# Note this requires an internet connection to load Google's JS library.
def get_json_for_word_tree(contact):
#     df = fully_merged_messages_df[(fully_merged_messages_df.full_name == contact)]
#     print('Exchanged {0:,} texts with {1}'.format(df.shape[0], contact))
    
    array_for_json = [[text[1]] for text in subdf.Text.iteritems()]
    array_for_json.insert(0, [['Phrases']])
    return json.dumps(array_for_json)
    
CONTACT_NAME = ''
ROOT_WORD = ''
HTML(wordtree.get_word_tree_html(get_json_for_word_tree(''),
                                 ROOT_WORD.lower(),
                                 lowercase=True,
                                 tree_type='double'))

### Create TFIDF matrix for all contacts

Note the methods below focus on texts received from these contacts, not texts you've sent to them.

In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
import numpy as np

vectorizer = TfidfVectorizer(preprocessor=df.Text.values,
                             tokenizer=tokenize.WordPunctTokenizer().tokenize,
#                              stop_words=processed_stopwords,
                             ngram_range=(1, 2), max_df=.9, max_features=50000)
tfidf_transformed_dataset = vectorizer.fit_transform(df.Text.values)
word_list = pd.Series(vectorizer.get_feature_names())

print('TFIDF sparse matrix is {0}MB'.format(tfidf_transformed_dataset.data.nbytes / 1024 / 1024))
print('TFIDF matrix has shape: {0}'.format(tfidf_transformed_dataset.shape))

TypeError: 'numpy.ndarray' object is not callable

### Helper methods to leverage the TFIDF matrix

In [ ]:
def get_word_summary_for_word(word, top_n=25):
    word = convert_unicode_to_str_if_needed(word)
    tfidf_record = _get_tfidf_record_for_word(word)
    if tfidf_record is None:
        print('"{0}" was not found.'.format(word))
        return
    sorted_indices = tfidf_record.argsort()[::-1]
    return pd.DataFrame({'Word': word_list.iloc[sorted_indices[:top_n]]}).reset_index(drop=True)

def get_word_summary_for_diffs(word, other_word, top_n=25):
    word = convert_unicode_to_str_if_needed(word)
    other_word = convert_unicode_to_str_if_needed(other_word)
    
    tfidf_record_word = _get_tfidf_record_for_word(word)
    tfidf_record_other_word = _get_tfidf_record_for_word(other_word)
    
    if tfidf_record_word is None or tfidf_record_other_word is None:
        # Print out the first word not found.
        word_not_found = word if tfidf_record_word is None else other_word
        print('"{0}" was not found.'.format(word_not_found))
        return
    sorted_indices = (tfidf_record_word - tfidf_record_other_word).argsort()[::-1]
    return pd.DataFrame({'Word': word_list.iloc[sorted_indices[:top_n]]}).reset_index(drop=True)

# Returns the row in the TFIDF matrix for a given word by name.
def _get_tfidf_record_for_word(word):
    if word not in grouped_by_name.index:
        return None
    row = np.argmax(grouped_by_name.index == word)
    return tfidf_transformed_dataset.getrow(row).toarray().squeeze()